In [1]:
# install api interface at https://github.com/cwendt94/ff-espn-api/

import requests

# get hardcoded cookies from browser


# helper function from ff-espn-api
def checkRequestStatus(status: int) -> None:
    if 500 <= status <= 503:
            raise Exception(status)
    if status == 401:
        raise Exception("Access Denied")

    elif status == 404:
        raise Exception("Invalid League")

    elif status != 200:
        raise Exception('Unknown %s Error' % status)


year = 2020
league_id = 83174673


# fetch league. need some data from cookies to fill in here
swid = '{9F7C3455-C43D-42A6-9D1A-AEB707CB5F0B}'
espn_s2 = 'AEAy3mkAv%2FsnK2YbgEVRFKpOw6%2ByQVeXH5BDBOlzgAiBg646ZtRuneInbY1z9QYrt1Ws8r6Cc8pS0g%2BVi7HmU0b%2F0lxgkP9csOvk9S01pusHVMJOZ9GfMZblZ01U9NjD46R2N%2BI21guim1Lga4TfVKqE6rCLMJvQszv5ZKdFgjngR98gxiClp9R3ZxzJ7cTLJufuir4JGnzKZ9RwOCL1%2BOaFy8qFEGwEVNGkuMY4Hj4zSR%2BL9a5fatMYEWNXvk4muhbgckHixSZiqvSnL6pyniu98SRFEpigMIkWgSJ%2FTfNvCg%3D%3D'
cookies = {
    'espn_s2': espn_s2,
    'SWID': swid
}
ENDPOINT = "https://fantasy.espn.com/apis/v3/games/FFL/seasons/" + str(year) + "/segments/0/leagues/" + str(league_id)
params = ''

# test request
r = requests.get(ENDPOINT, params=params, cookies=cookies)
status = r.status_code
checkRequestStatus(status)
data = r.json()
print(data)

{'gameId': 1, 'id': 83174673, 'members': [{'displayName': 'espn04014086', 'id': '{23752840-5024-4BC3-B528-4050243BC3EE}', 'isLeagueManager': False}, {'displayName': 'YungFly', 'id': '{2A52739B-A4D2-440F-B78D-C1282294B36E}', 'isLeagueManager': False}, {'displayName': 'Ball, call, install', 'id': '{5E1655AA-B7E8-4112-8208-4024E2B806B4}', 'isLeagueManager': False}, {'displayName': 'espn57414091', 'id': '{86F13096-2F1D-44BC-B130-962F1DA4BCEA}', 'isLeagueManager': False}, {'displayName': 'broncoz16', 'id': '{89BABC79-3E89-4484-B7E0-91FE30264A36}', 'isLeagueManager': False}, {'displayName': 'typhe12', 'id': '{8BCE107A-C396-4216-A165-204DCBB7ACED}', 'isLeagueManager': False}, {'displayName': 'ESPNFAN6778113520', 'id': '{9DAF6913-3970-416F-BECB-D0DE77788ADB}', 'isLeagueManager': False}, {'displayName': 'shottysnipesonthepit', 'id': '{9F7C3455-C43D-42A6-9D1A-AEB707CB5F0B}', 'isLeagueManager': False}, {'displayName': 'JackMichaels55', 'id': '{AED6F6FD-B3C1-49EC-9329-C5E971023ED7}', 'isLeagueMana

In [ ]:
"""
# fetch players, get general data about players, don't really remember what's in here
# this will print all players with last name starting with "a"s
params = {
    'scoringPeriodId': 3,
    'view': 'players_wl',
}
endpoint = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + str(year) + '/players'
r = requests.get(endpoint, params=params, cookies=cookies)
status = r.status_code
checkRequestStatus(status)
data = r.json()
print(data)
"""

In [29]:
import numpy as np

# fetch players, get scoring data
def get_player_data(week):
    params = {
        'scoringPeriodId': week
    }
    endpoint = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + str(year) + '/segments/0/leagues/' + str(league_id) + '?view=mMatchup&view=mMatchupScore'
    r = requests.get(endpoint, params=params, cookies=cookies)
    status = r.status_code
    checkRequestStatus(status)
    d = r.json()
    return d

# get player scoring data for all weeks
import pandas as pd

slotcodes = {
    0 : 'QB', 2 : 'RB', 4 : 'WR',
    6 : 'TE', 16: 'Def', 17: 'K',
    20: 'Bench', 21: 'IR', 23: 'Flex'
}

# holder variable
data = []

# iterate teams
for week in range(1, 14):

    # get player data for that scoring period
    d = get_player_data(week)

    for tm in d['teams']:
        tmid = tm['id']

        # iterate players on teams
        for p in tm['roster']['entries']:
            name = p['playerPoolEntry']['player']['fullName']
            slot = p['lineupSlotId']
            pos  = slotcodes[slot]

            # injured status (need try/exc bc of D/ST)
            inj = 'NA'
            try:
                inj = p['playerPoolEntry']['player']['injuryStatus']
            except:
                pass

            # grab projected/actual points
            proj, act = None, None
            for stat in p['playerPoolEntry']['player']['stats']:
                if stat['scoringPeriodId'] != week:
                    continue
                if stat['statSourceId'] == 0:
                    act = stat['appliedTotal']
                elif stat['statSourceId'] == 1:
                    proj = stat['appliedTotal']

            data.append([
                week, tmid, name, slot, pos, inj, proj, act
            ])
    print('Week {} complete.'.format(week))

data = pd.DataFrame(data, columns=['Week', 'Team', 'Player', 'Slot', 'Pos', 'Status', 'Proj', 'Actual'])
#


Week 1 complete.
Week 2 complete.
Week 3 complete.
Week 4 complete.
Week 5 complete.
Week 6 complete.
Week 7 complete.
Week 8 complete.
Week 9 complete.
Week 10 complete.
Week 11 complete.
Week 12 complete.
Week 13 complete.


In [32]:
# load data from csv
#data.to_csv('20201209_data.csv')
data = pd.read_csv('20201209_data.csv')

In [33]:
# let's focus on player data not on bench
starter_data = data[data['Pos'] != 'Bench']

,Unnamed: 0,Week,Team,Player,Slot,Pos,Status,Proj,Actual
150,150,2,1,Clyde Edwards-Helaire,23,Flex,ACTIVE,16.530201,13.00
151,151,2,1,Aaron Jones,2,RB,ACTIVE,17.491481,45.60
152,152,2,1,DJ Moore,4,WR,OUT,13.564116,20.00
153,153,2,1,Cooper Kupp,4,WR,ACTIVE,13.603843,13.00
155,155,2,1,Todd Gurley II,2,RB,QUESTIONABLE,14.200670,6.10
...,...,...,...,...,...,...,...,...,...
292,292,2,10,Robert Woods,4,WR,ACTIVE,14.477450,11.30
294,294,2,10,Matt Ryan,0,QB,ACTIVE,18.391818,28.52
295,295,2,10,Bills D/ST,16,Def,NaN,9.194751,-1.00
298,298,2,10,Robbie Gould,17,K,ACTIVE,7.800011,8.00


In [153]:
# convenience functions
def get_player_performance_rank(player, week, only_starters=True):
    """ function that evaluates the positional ranking of a player among starters that week
    I think this metric has better insight into how good a player is """

    # load data
    data = pd.read_csv('20201209_data.csv')
    if only_starters:
        data = data[data['Pos'] != 'Bench']
    
    # just work with data for that week
    week_data = data[data['Week'] == week]

    # get position of that player
    pos = week_data[week_data['Player'] == player]['Pos']

    # return nan if player didn't start that week
    # and the person had "only_starters" selected
    if pos.empty:
        print("Warning: player {} not in dataset for week {}. Returning NaN.".format(player, week))
        return np.NaN   

    # get scores of players of that position
    pos_data = week_data[week_data['Pos'] == pos.unique()[0]]

    # get scores and sort descending
    scores = pos_data['Actual'].unique()
    scores.sort()
    scores = scores[::-1]

    # get where player score is in rank of scores
    player_score = pos_data[pos_data['Player']==player]['Actual'].tolist()[0]
    rank = np.where(scores == player_score)[0][0]

    # increment cause zero indexing
    rank += 1

    return rank


def get_player_score(player, week):
    """ Convenience function to get score of a player given data and week
    """
    data = pd.read_csv('20201209_data.csv')
    data = data[data['Week'] == week]

    # get data for that player
    player_data = data[data['Player'] == player]

    # return nan if player didn't score in that dataset for whatever reason
    if player_data.empty:
        print("Warning: no player score for {} recorded for week {}.".format(player, week))
        return np.NaN

    # return score
    player_score = player_data['Actual'].tolist()[0]

    return player_score


def was_player_started(player, week):

    # just work with data for that week
    data = pd.read_csv('20201209_data.csv')
    data = data[data['Week'] == week]
    player_data = data[data['Player'] == player]

    # if player wasn't in dataset, he was in waivers
    if player_data.empty:
        return False

    # if player was in dataset as bench, he was on bench
    if player_data['Pos'] == 'Bench':
        return False

    # if not the above, then ya
    return True


def was_player_on_bench(player, week):
    # just work with data for that week
    data = pd.read_csv('20201209_data.csv')
    data = data[data['Week'] == week]
    player_data = data[data['Player'] == player]

    # if player wasn't in dataset, he was in waivers
    if player_data.empty:
        return False

    # if player was in dataset as bench, he was on bench
    if player_data['Pos'] == 'Bench':
        return True

    # otherwise he started
    else:
        return False


def was_player_on_waivers(player, week):
    # just work with data for that week
    data = pd.read_csv('20201209_data.csv')
    data = data[data['Week'] == week]
    player_data = data[data['Player'] == player]

    # if player wasn't in dataset, he was in waivers
    if player_data.empty
        return True
    else:
        return False

SyntaxError: invalid syntax (<ipython-input-153-39b460046bb4>, line 69)

In [152]:
player = 'Amari Cooper'
week = 1
rank = get_player_performance_rank(player=player, week=week)
rank

10